In [16]:
import cv2
import pandas as pd
from ultralytics import YOLO
from moviepy import VideoFileClip, concatenate_videoclips
import numpy as np
import os

# --- KONFIGŪRACIJA ---
VIDEO_PATH = "data/Lakers_vs_Thunder.mp4"
OUTPUT_FOLDER = "results"
OUTPUT_PATH = os.path.join(OUTPUT_FOLDER, "summary.mp4")
CSV_PATH = os.path.join(OUTPUT_FOLDER, "analysis_data.csv")
MODEL_NAME = "models/yolo11s.pt"
TARGET_DURATION = 120
SCAN_INTERVAL = 10

In [17]:
def analyze_video(video_path, model):
    if not os.path.exists(video_path):
        raise FileNotFoundError(f"KLAIDA: Video failas nerastas! Tikrinta vieta: {os.path.abspath(video_path)}")

    print(f"Pradedama video analizė: {video_path}")
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        raise ValueError("KLAIDA: OpenCV negali atidaryti video failo. Failas gali būti sugadintas arba netinkamo formato.")

    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    yolo = YOLO(model)
    data = []

    frame_idx = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        if frame_idx % SCAN_INTERVAL == 0:
            results = yolo(frame, verbose=False, classes=[0, 32])

            person_count = 0
            ball_detected = 0

            for r in results:
                boxes = r.boxes
                for box in boxes:
                    cls = int(box.cls[0])
                    if cls == 0:
                        person_count += 1
                    elif cls == 32:
                        ball_detected = 1

            score = (person_count * 1) + (ball_detected * 50)

            data.append({
                'frame': frame_idx,
                'time_sec': frame_idx / fps,
                'score': score
            })

            # Spausdiname progresą kas 100-ąjį nuskaitytą kadrą
            if len(data) % 10 == 0:
                print(f"Progresas: {frame_idx}/{total_frames} | Balas: {score}...", end='\r')

        frame_idx += 1

    cap.release()
    df = pd.DataFrame(data)
    print("\nAnalizė baigta.")
    return df, fps


In [18]:
def create_summary(video_path, df, fps, target_duration):
    if df.empty:
        print("KLAIDA: Jokie duomenys nebuvo surinkti. Video generavimas negalimas.")
        return

    print("Generuojamas montažas...")

    # 1. Paruošiame duomenis
    df['sec_int'] = df['time_sec'].astype(int)
    sec_scores = df.groupby('sec_int')['score'].mean().reset_index()

    # 2. Išrenkame geriausias sekundes
    top_seconds = sec_scores.sort_values(by='score', ascending=False).head(target_duration)
    top_seconds = top_seconds.sort_values(by='sec_int')

    # 3. Sujungiame sekundes į intervalus
    clips_intervals = []
    if not top_seconds.empty:
        times = top_seconds['sec_int'].tolist()
        start = times[0]
        prev = times[0]

        for t in times[1:]:
            if t > prev + 1:
                clips_intervals.append((start, prev + 1))
                start = t
            prev = t
        clips_intervals.append((start, prev + 1))

    original_clip = VideoFileClip(video_path)
    subclips = []

    print(f"Rasta {len(clips_intervals)} svarbių epizodų.")

    for start, end in clips_intervals:
        end_time = min(original_clip.duration, end)
        start_time = max(0, start)

        if start_time < end_time:
            subclip = original_clip.subclipped(start_time, end_time)
            subclips.append(subclip)

    if subclips:
        final_clip = concatenate_videoclips(subclips)
        final_clip.write_videofile(OUTPUT_PATH, codec="libx264", audio_codec="aac")
        print(f"Rezultatas sėkmingai išsaugotas: {OUTPUT_PATH}")
    else:
        print("Nepavyko sugeneruoti klipų (galbūt per trumpas video?).")

In [19]:
if __name__ == "__main__":
    if not os.path.exists(OUTPUT_FOLDER):
        os.makedirs(OUTPUT_FOLDER)
        print(f"Sukurtas aplankas: {OUTPUT_FOLDER}")

    try:
        df_scores, fps_rate = analyze_video(VIDEO_PATH, MODEL_NAME)

        # Išsaugome CSV
        df_scores.to_csv(CSV_PATH, index=False)
        print(f"Duomenys išsaugoti: {CSV_PATH}")

        create_summary(VIDEO_PATH, df_scores, fps_rate, TARGET_DURATION)

    except FileNotFoundError as e:
        print("\n!!! SUSTOTA DĖL KLAIDOS !!!")
        print(e)
        print("PATARIMAS: Patikrinkite, ar failo pavadinimas 'VIDEO_PATH' kintamajame sutampa su failu 'data' aplanke.")
    except Exception as e:
        print(f"Įvyko nenumatyta klaida: {e}")

Pradedama video analizė: data/Lakers_vs_Thunder.mp4
Progresas: 35490/35519 | Balas: 50...
Analizė baigta.
Duomenys išsaugoti: results/analysis_data.csv
Generuojamas montažas...
Rasta 73 svarbių epizodų.
MoviePy - Building video results/summary.mp4.
MoviePy - Writing audio in summaryTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
MoviePy - Writing video results/summary.mp4



frame_index: 100%|█████████▉| 7168/7170 [01:36<00:00, 166.06it/s, now=None]/Users/paulius/Documents/VGTU/Magistras/vaizdu_analize/videoRecogYOLO/.venv/lib/python3.13/site-packages/moviepy/video/io/ffmpeg_reader.py:190: UserWarning: In file data/Lakers_vs_Thunder.mp4, 6220800 bytes wanted but 0 bytes read at frame index 35519 (out of a total 35521 frames), at time 592.58/592.62 sec. Using the last valid frame instead.
  warnings.warn(
/Users/paulius/Documents/VGTU/Magistras/vaizdu_analize/videoRecogYOLO/.venv/lib/python3.13/site-packages/moviepy/video/io/ffmpeg_reader.py:190: UserWarning: In file data/Lakers_vs_Thunder.mp4, 6220800 bytes wanted but 0 bytes read at frame index 35520 (out of a total 35521 frames), at time 592.59/592.62 sec. Using the last valid frame instead.
  warnings.warn(


MoviePy - Done !
MoviePy - video ready results/summary.mp4
Rezultatas sėkmingai išsaugotas: results/summary.mp4
